# YOLO5 

In [ ]:
# install cv2
# this might be quicker from the cli
!apt-get update && apt-get install ffmpeg libsm6 libxext6 -y
!pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org opencv-python opencv-python-headless

In [ ]:
# import the libraries
import os
import glob as glob
import matplotlib.pyplot as plt
#import cv2
import requests

In [ ]:
# Define the hyperparameter values
TRAIN = True

# Number of epochs to train for 
EPOCHS = 5

In [ ]:
# Helper function to download files
def download(url, save_name):
    url = url
    if not os.path.exists(save_name):
        file = requests.get(url)
        open(save_name, 'wb').write(file.content)
    else:
        print("file already exists, skipping download...")

In [ ]:
# Pull images from roboflow
# data link : "https://public.roboflow.com/ds/xKV14HbTF?key=aJzo7msVta"

if not os.path.exists('train'):
    !/usr/bin/curl -L "https://public.roboflow.com/ds/xKV14HbTF?key=aJzo7msVta" > roboflow.zip; unzip roboflow.zip; 
    !rm roboflow.zip

    dirs = ['train','valid','test']

    for i, dir_name in enumerate(dirs):
        all_image_names = sorted(os.listdir(f"{dir_name}/images/"))
        for j, image_name in enumerate(all_image_names):
            if(j % 2) == 0:
                file_name = image_name.split('.jpg')[0]
                os.remove(f"{dir_name}/images/{image_name}")
                os.remove(f"{dir_name}/labels/{file_name}.txt")

In [ ]:
# clone the yolo5 repository
# link : !git clone https://github.com/ultralytics/yolo5.git

if not os.path.exists('yolov5'):
    !git clone https://github.com/ultralytics/yolov5.git

%cd yolov5/
    
!pip install -r requirements.txt

In [ ]:
# directory to store results
def set_res_dir():
    # dri
    res_dir_count = len(glob.glob('runs/train/*'))
    print(f"Current number of result directories: {res_dir_count}")
    if TRAIN:
        RES_DIR = f"results_{res_dir_count+1}"
        print(RES_DIR)
    else:
        RES_DIR = f"results_{res_dir_count}"
    return RES_DIR


In [ ]:
# Running the model
RES_DIR = set_res_dir()
if TRAIN:
    # VDC: python3
    !python train.py --data ../data.yaml --weights yolov5s.pt \
    --img 640 --epochs {EPOCHS} --batch-size 16 --name {RES_DIR}

## Object Detection

In [ ]:
# Function to show validation predictions saved during training
def show_valid_results(RES_DIR):
    !ls runs/train/{RES_DIR}
    EXP_PATH = f" runs/train{RES_DIR}"
    validation_pred_images = glob.glob(f"{EXP_PATH}/*_pred.jpg")
    print(validation_pred_images)
    for pred_image in validation_pred_images:
        image = cv2.imread(pred_image)
        plt.figure(figsize=(19,16))
        plt.imshow(image[:,:,::-1])
        plt.axis('off')
        plt.show()

In [ ]:
# helper function for inference on images
def inference(RES_DIR, data_path):
    infer_dir_count = len(glob.glob('runs/detect/*'))
    print(f" Current number of inference detection directories:{infer_dir_count}")
    INFER_DIR = f"inference_{infer_dir_count+1}"
    print(INFER_DIR)

    # Inference on images
    !python detect.py --weights runs/train{RES_DIR}/weights/best.pt\
    --source {data_path} --name {INFER_DIR}

    return INFER_DIR

In [ ]:
# Visualize inference images
def visualise (INFER_DIR):
    # visualize inference images
    INFER_PATH = f"runs/detect/{INFER_DIR}"
    infer_images = glob.glob(f"{INFER_PATH}/*.jpg")
    print(infer_images)
    for pred_image in infer_images:
        image = cv2.imread(pred_image)
        plt.figure(figsize=(19,16))
        plt.imshow(image[:,:,::-1])
        plt.axis('off')
        plt.show()

In [ ]:
# show results
show_valid_results()

In [ ]:
# Downloading new files for inference
download_file("https://learnopencv.s3.us-west-2.amazonaws.com/yolov5_inference_data.zip",
              "inference_data.zip")

if not os.path.exists("inference_images"):
    !unzip -q "inference_data.zip"
else:
    print("dataset already present")

In [ ]:
IMAGE_INFER_DIR = inference(RES_DIR, 'inference_images')

In [ ]:
# visualize inference
visualize(IMAGE_INFER_DIR)